In [2]:
from dputils import scrape as sc
from bs4 import BeautifulSoup

In [3]:
soup = sc.get_webpage_data('https://www.amazon.in/Apple-MacBook-Chip-13-inch-512GB/dp/B08N5YD6NF/ref=sr_1_1?crid=JK07X5YJ2WWY&keywords=laptop&qid=1661143760&sprefix=laptop%2Caps%2C265&sr=8-1')

In [4]:
review_link_area = soup.find('div',attrs={'id':'reviews-medley-footer'})
all_review_link = review_link_area.find('a').get('href')
print(all_review_link)

/Apple-MacBook-Chip-13-inch-512GB/product-reviews/B08N5YD6NF/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews


In [5]:
final_url = 'https://www.amazon.com' + all_review_link
print(final_url)

https://www.amazon.com/Apple-MacBook-Chip-13-inch-512GB/product-reviews/B08N5YD6NF/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews


In [6]:
def get_review_link(product_url="", base_url='https://www.amazon.com'):
    try:
        soup = sc.get_webpage_data(product_url)
        review_link_area = sc.extract_one(soup, div={'tag':'div','attrs':{'id':'reviews-medley-footer'},'output':'object'})
        all_review_link = review_link_area['div'].find('a').get('href')
        return base_url + all_review_link
    except Exception as e:
        print(e)
        return None


In [31]:
target = {'tag':'div','attrs':{'id':'cm_cr-review_list'}} # the page subpart
items= {'tag':'div','attrs':{'class':'a-section review aok-relative'}} # the repeating elements
reviewer = {'tag':'span','attrs':{'class':'a-profile-name'}} # the name inside a single repeating element
review = {'tag':'span','attrs':{'class':'a-size-base review-text review-text-content'}} # the review inside a single repeating element

In [43]:
page = 1
product_url = 'https://www.amazon.com/Apple-MacBook-13-inch-256GB-Storage/dp/B08N5M7S6K/ref=sr_1_1?crid=3D2TWY8NWW02P&keywords=macbook&qid=1661143915&sprefix=macbook%2Caps%2C348&sr=8-1'
all_reviews_link = get_review_link(product_url)
datalist = []
limit = 0
while True:
    review_link = all_reviews_link + '&pageNumber=' + str(page)
    print('=>>>',review_link)
    review_soup = sc.get_webpage_data(review_link)
    output =sc.extract_many(review_soup, target=target, items=items, reviewer=reviewer, content= review)
    if output is None:
        print("LOG: No more reviews, output is None")
        break
    elif len(output) == 0:
        print("LOG: No more reviews, output is empty")
        break
    elif limit > 100:
        print(f"LOG: No more reviews, limit is {limit}")
        break
    else:
        datalist.extend(output)
        page += 1
        limit += 1
print(len(datalist))

=>>> https://www.amazon.com/Apple-MacBook-13-inch-256GB-Storage/product-reviews/B08N5M7S6K/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
10 items found
All items extracted
=>>> https://www.amazon.com/Apple-MacBook-13-inch-256GB-Storage/product-reviews/B08N5M7S6K/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=2
10 items found
All items extracted
=>>> https://www.amazon.com/Apple-MacBook-13-inch-256GB-Storage/product-reviews/B08N5M7S6K/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=3
10 items found
All items extracted
=>>> https://www.amazon.com/Apple-MacBook-13-inch-256GB-Storage/product-reviews/B08N5M7S6K/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=4
10 items found
All items extracted
=>>> https://www.amazon.com/Apple-MacBook-13-inch-256GB-Storage/product-reviews/B08N5M7S6K/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=5
10 items found
All items extracted


In [44]:
len(datalist)

1010

In [45]:
import pandas as pd

In [46]:
pd.DataFrame(datalist)

,reviewer,content
0,K. MURPHY,"UPDATE 12-15-2020 - After three weeks, I'm sti..."
1,Try it or DIY it,Beautiful laptop. I’ve upgraded from the MacBo...
2,Impetigo,"I have two MacBook Airs (MBA), a 13"" and a 11,..."
3,Paul K.,Liked:- Long battery life (one charge WILL las...
4,Adi,I'm a fan of apple products and I like almost ...
...,...,...
1005,lynellw,Great purchase
1006,arlene r.,My first Apple product. So far finding it diff...
1007,Gautam,Product was broken
1008,Jus me,My child is taking class on this Laptop
